In [ ]:
###Run only if Colab or else install these seperately###

!pip install pytesseract 
!pip install pdf2image
!apt-get install poppler-utils 
#!pip install poppler-utils
!sudo apt install tesseract-ocr

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=63cdc83bdcda1ef6fd10ec28c3f1eb317faf47ef93ac11769b8e3ce31a4d93e9
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (280 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160706 files and di

In [ ]:
!pip install networkx

# Summarization

In [ ]:
import numpy as np
import pandas as pd
import nltk
import networkx as nx
nltk.download('punkt') # one time execution
nltk.download('stopwords')
import re

from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def convert_to_string(path_of_file):
  pages = convert_from_path(path_of_file) 
  rawText = ""

  for page in pages:
    text = str(((pytesseract.image_to_string(page))))
    text = text.replace('-\n', '')
    rawText += text

  with open("output.txt", "w") as f:
    f.write(rawText)

  return rawText


In [ ]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
def preprocessing(pdf_file):
  rawList = pdf_file.splitlines()
  rawList = [item for item in rawList if item.strip()]
  rawList_str = ' '.join(rawList)

  from nltk.tokenize import sent_tokenize
  sentences = []
  for s in rawList_str.splitlines():
    sentences.append(sent_tokenize(s))

  sentences = [y for x in sentences for y in x]

  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

  # make alphabets lowercase
  clean_sentences = [s.lower() for s in clean_sentences]

  clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
  #print(clean_sentences)

  return clean_sentences, sentences

In [ ]:
def compute_vectors(sentences):

  word_embeddings = {}
  f = open('/content/drive/MyDrive/glove.6B.100d.txt', encoding='utf-8')
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
  f.close()

  sentence_vectors = []
  for i in sentences:
    if len(i) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)

  return sentence_vectors

In [ ]:
def textrank(path_to_file):

  doc_string = convert_to_string(path_to_file)

  clean_sentences, sentences = preprocessing(doc_string)

  sentence_vectors = compute_vectors(clean_sentences)

  sim_mat = np.zeros([len(sentences), len(sentences)])
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)

  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  return ranked_sentences


In [ ]:
days = input("Number of sentences in the summary: ")
file_path = input("Enter the path of the file")

ranked_sentences = textrank(file_path)
for i in range(int(days)):
  print(ranked_sentences[i][1])

Number of sentences in the summary: 10
Enter the path of the file/content/drive/MyDrive/2021-02-08_104743.pdf


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (150876400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


To maximize absorption and clinical benefit, Idrofos” 150 should be taken after an overnight fast (atleast 6 hours) and at least 60 minutes before the first food or drink (other than water) of the day or before taking any oral medication or supplementation, including calcium, antacids, or vitamins.
The observed apparent terminal half-life for the 150 mg ibandronic acid tablet upon oral administration to healthy postmenopausal women ranges from 37 to 157 hours.
Since ibandronic acid is not known to be metabolised, the only difference in ibandronic acid elimination for geriatric patients versus younger patients is expected to relate to progressive age related changes in renal function.
However, voth bioavailability and the effect on bone mineral density (BMD) are reduced when food or beverages are taken less than 60 minutes following an ibandronic acid dose.
Two patients who received ibandronic acid monthly experienced ocular inflammation, one was a case of uveitis and the other scleriti

# POS Tagging (Syntax analysis)

In [ ]:
doc_string = convert_to_string("/content/drive/MyDrive/2021-02-08_104743.pdf")
clean_doc, sentences = preprocessing(doc_string)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (150876400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [ ]:
clean_doc

['use registered medical practitioner hospital laboratory',
 'prescribing information inandronic acid tablets idrofos dosage form tablets composition film coated tablet contains lbandronic acid monosodium monohydrate equivalentto ibandronic acid mg excipients clinical pharmacology lbandronic acid anitrogen containing bisphosphonate inhibits osteoclast mediated bone resorption',
 'mechanism action action ibandronic acid bone tissue based affinity hydroxyapatite part mineral matrix bone',
 'ibandronic acid inhibits osteoclast activity reduces bone resorption turnover',
 'post menopausal women reduces elevated rate bone turnover leading average net gain bone mass',
 'pharmacokinetics absorption oral ibandronic acid occurs upper gastrointestinal tract',
 'plasma concentrations increase ina dose linear manner mg oral intake increases nonlinearly dose',
 'following oral dosing time maximum observed plasma ibandronic acid concentrations ranged hours median hour fasted healthy postmenopausal w

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
clean_doc = ". ".join(clean_doc) 
doc = nlp(clean_doc)

In [ ]:
from collections import Counter

c = Counter(([token.pos_ for token in doc]))
sbase = sum(c.values())
for el, cnt in c.items():
    print(el, '{0:2.2f}%'.format((100.0* cnt)/sbase))

NOUN 38.01%
VERB 17.28%
PROPN 15.08%
PUNCT 8.44%
ADJ 14.88%
ADP 0.80%
ADV 3.06%
CCONJ 0.07%
X 0.27%
NUM 0.86%
DET 0.53%
SCONJ 0.53%
PRON 0.13%
INTJ 0.07%


In [ ]:

#raw_sentences = ". ".join(doc_string) 
raw_doc = nlp(doc_string)

In [ ]:
import random
import os
from pathlib import Path
from spacy import displacy
num = random.randint(0,100)
print(num)
for i,sent in enumerate(raw_doc.sents):
  if i == num:
    print(sent)
    svg = displacy.render(sent, style="dep", jupyter= False, minify = True)
    output_path = Path(os.path.join("./", "sentence.svg"))
    output_path.open('w', encoding="utf-8").write(svg)
    break

60
For patients who develop osteonecrosis of the jaw (ONJ) while on bisphosphonate therapy, dental surgery may
exacerbate the condition.




In [ ]:
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]

In [ ]:
word_freq = Counter(words)
common_words = word_freq.most_common(8)

In [ ]:
common_word_list = [i[0] for i in common_words]
common_word_list

['acid', 'ibandronic', 'patients', 'idrofos', 'dose', 'bone', 'tablet', 'day']



# WORDNET (Semantic Analysis)


In [ ]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
syns = wn.synsets('acid')
syns2 = wn.synsets('patients')

In [ ]:
for word in common_word_list:
  syns = wn.synsets(word)
  for i in syns:
    print(i.lemmas()[0].name() + " ("+ i.name() + ") :")
    #x = wn.synset(i)
    print("Definition: " + i.definition())
    if len(i.examples()) > 0:
      print("Example: " + i.examples()[0])
    print(f"Lemmas: {[str(lemma.name()) for lemma in i.lemmas()]}")
    #print(f"Synonyms: {[str(synonym.name()) for synonym in i.synonyms()]}")
    print(f"Hypernyms: {[str(hypernym.name()) for hypernym in i.hypernyms()]}")
    print(f"Hyponyms: {[str(hyponym.name()) for hyponym in i.hyponyms()]}")
    print(f"Hyponyms: {[str(polysem.name()) for polysem in i.part_meronyms()]}")
    print()
    print()

acid (acid.n.01) :
Definition: any of various water-soluble compounds having a sour taste and capable of turning litmus red and reacting with a base to form a salt
Lemmas: ['acid']
Hypernyms: ['compound.n.02']
Hyponyms: ['alkapton.n.01', 'aminobenzoic_acid.n.01', 'aqua_regia.n.01', 'arsenic_acid.n.01', 'barbituric_acid.n.01', 'boric_acid.n.01', 'boric_acid.n.02', 'bromic_acid.n.01', 'carbamic_acid.n.01', 'carbolic_acid.n.01', 'carbonic_acid.n.01', 'carboxylic_acid.n.01', 'cerotic_acid.n.01', 'chloric_acid.n.01', 'chlorous_acid.n.01', 'chromic_acid.n.01', 'citric_acid.n.01', 'cyanamide.n.01', 'cyanic_acid.n.01', 'cyanuric_acid.n.01', 'dibasic_acid.n.01', 'dichromic_acid.n.01', 'ferricyanic_acid.n.01', 'ferrocyanic_acid.n.01', 'fluoroboric_acid.n.01', 'fluosilicic_acid.n.01', 'formic_acid.n.01', 'fulminic_acid.n.01', 'fumaric_acid.n.01', 'gallic_acid.n.01', 'gamma_acid.n.01', 'glyceric_acid.n.01', 'glycolic_acid.n.01', 'hydriodic_acid.n.01', 'hydrobromic_acid.n.01', 'hydrochloric_acid.n.

In [ ]:
synonyms = []
antonyms = []

for syn in wn.synsets("acid"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(set(synonyms))
print(set(antonyms))

{'dose', 'Zen', 'sulfurous', 'acidulent', 'battery-acid', 'caustic', 'Elvis', 'acidulous', 'acrid', 'blistering', 'superman', 'acid', 'bitter', 'vitriolic', 'pane', 'loony_toons', 'virulent', 'sulphurous', 'Lucy_in_the_sky_with_diamonds', 'acerb', 'back_breaker', 'dot', 'window_pane', 'acerbic', 'acidic'}
set()


In [ ]:
w1 = wn.synset('acid.n.01')
w2 = wn.synset('patient.n.01')
print(w1.wup_similarity(w2))

In [ ]:
from itertools import combinations

In [ ]:
word_comb = combinations(common_word_list,2)
for i in word_comb:
  print(i[0])

In [ ]:
for i in combinations(common_word_list,2):
  
  a = wn.synsets(i[0])
  b = wn.synsets(i[1])

  if len(a)>0 and len(b)>0:
    print(f"Path similarity between {i[0]} and {i[1]} = {a[0].path_similarity(b[0])}")
  

Path similarity between acid and patients = 0.09090909090909091
Path similarity between acid and dose = 0.09090909090909091
Path similarity between acid and bone = 0.07142857142857142
Path similarity between acid and tablet = 0.07692307692307693
Path similarity between acid and day = 0.09090909090909091
Path similarity between patients and dose = 0.125
Path similarity between patients and bone = 0.08333333333333333
Path similarity between patients and tablet = 0.1
Path similarity between patients and day = 0.1
Path similarity between dose and bone = 0.07692307692307693
Path similarity between dose and tablet = 0.08333333333333333
Path similarity between dose and day = 0.09090909090909091
Path similarity between bone and tablet = 0.07142857142857142
Path similarity between bone and day = 0.07692307692307693
Path similarity between tablet and day = 0.08333333333333333
